# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
EBOLA = DATA_FOLDER + "/ebola"
GUINEA = EBOLA + "/guinea_data"
LIBERIA = EBOLA + "/liberia_data"
SIERRA_LEONE = EBOLA + "/sl_data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here
import pandas as pd
import glob

def create_folder_data_frame(FOLDER):
    list_ = []
    for file_ in glob.glob(FOLDER + "/*.csv"):
        df = pd.read_csv(file_, index_col=None)
        list_.append(df)
    return pd.concat(list_)

def index_and_country(data_frame, country):
    data_frame["Country"] = country
    data_frame.set_index(['Date', 'Description', 'Country'], inplace=True)
    return data_frame
    
def preprocess_guinea(FOLDER):
    guinea_frame = create_folder_data_frame(FOLDER)
    guinea_frame.Date = pd.to_datetime(guinea_frame.Date)
    guinea_frame = index_and_country(guinea_frame, "Guinea")
    
    return guinea_frame
    
preprocessed_guinea = preprocess_guinea(GUINEA)

# Columns not towns: Date, Description Totals
preprocessed_guinea

In [ ]:
def preprocess_liberia(FOLDER):
    liberia_frame = create_folder_data_frame(FOLDER)
    liberia_frame.rename(columns={"Variable": "Description", "National": "Totals", 'Unnamed: 18': "Unknown"}, inplace=True)
    liberia_frame.Date = pd.to_datetime(liberia_frame.Date)
    liberia_frame = index_and_country(liberia_frame, "Liberia")
    
    return liberia_frame

preprocessed_liberia = preprocess_liberia(LIBERIA)
preprocessed_liberia

In [ ]:
def preprocess_sierra(FOLDER):
    sierra_frame = create_folder_data_frame(FOLDER)
    sierra_frame.rename(columns={"variable": "Description", "National": "Totals", 'Unnamed: 18': "Unknown", "date": "Date"}, inplace=True)
    sierra_frame.Date = pd.to_datetime(sierra_frame.Date)
    sierra_frame = index_and_country(sierra_frame, "Sierra Leone")
    
    return sierra_frame





In [ ]:
preprocessed_sierra = preprocess_sierra(SIERRA_LEONE)

#Set all values to integer and replace NaN by 0.
preprocessed_sierra=preprocessed_sierra[:].apply(pd.to_numeric,errors='coerce')
preprocessed_sierra = preprocessed_sierra[:].fillna(0.0).astype(int)

#Keep descriptions with interest for us
description = ['death_suspected','new_probable','new_suspected','death_confirmed','new_confirmed','death_probable']
ix=preprocessed_sierra.index.get_level_values('Description').isin(description)
preprocessed_sierra = preprocessed_sierra[ix]

#Create new DataFrame with descriptions as columns
sierra = pd.DataFrame()
for cat in description:
    sierra[cat] = preprocessed_sierra.xs((cat,'Sierra Leone'),level=('Description','Country'))['Totals'].tolist()

#The data give us only the cumulated total for the deaths. So we computed the dayly numbers of deaths
deaths = ['death_suspected','death_probable','death_confirmed']
for c in sierra.columns:
    if c in deaths:
        sierra[c] = sierra[c] - sierra[c][0]
        sierra[c][1:] = sierra[c][1:].copy().as_matrix() - sierra[c][:-1].copy().as_matrix()

#Set the minimum value to 0    
sierra = sierra.clip(lower=0)

#Set the country and the date as index
sierra['Country'] = 'Sierra Leone'
date = sorted(list(set(preprocessed_sierra.index.get_level_values('Date'))))
sierra['Date'] = date
sierra.set_index(['Date','Country'],inplace=True)
sierra


In [ ]:
#Compute the mean by month and country
#sierra = sierra.groupby([sierra.index.get_level_values('Date').month,sierra.index.get_level_values('Country')]).mean()




In [ ]:
sierra_descriptions = set([b for a,b,c, in list(preprocessed_sierra.index.values)])
print("Sierra: {}\n".format(sierra_descriptions))

guinea_descriptions = set([b for a,b,c, in list(preprocessed_guinea.index.values)])
print("Guinea: {}\n".format(guinea_descriptions))

liberia_descriptions = set([b for a,b,c, in list(preprocessed_liberia.index.values)])
print("Liberia: {}\n".format(liberia_descriptions))
#sierra[~sierra['34 Military Hospital'].isnull()]
#preprocessed_sierra.iloc[0]


Guinea: Total new cases registered so far
        

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here